In [1]:
import math

### Define computational relations

In [60]:
def fangle(U):
    A = U.get("A", None)
    B = U.get("B", None)
    C = U.get("C", None)

    if A is None and B is not None and C is not None:
        return {"A": math.pi - (B + C)}, ["B", "C"]
    if B is None and A is not None and C is not None:
        return {"B": math.pi - (A + C)}, ["A", "C"]
    if C is None and A is not None and B is not None:
        return {"C": math.pi - (A + B)}, ["A", "B"]
    return {}, []
    
def fsin(U, edges, angles):
    a = U.get(edges[0], None)
    b = U.get(edges[1], None)
    A = U.get(angles[0], None)
    B = U.get(angles[1], None)

    if a is None and b is not None and A is not None and B is not None:
        return {edges[0]: (math.sin(A) * b) / math.sin(B)}, [edges[1], angles[0], angles[1]]
    if b is None and a is not None and A is not None and B is not None:
        return {edges[1]: (math.sin(B) * a) / math.sin(A)}, [edges[0], angles[0], angles[1]]
    
    if A is None and a is not None and b is not None and B is not None:
        sinA = a * math.sin(B) / b
        return {angles[0]: math.asin(sinA)}, [edges[0], edges[1], angles[1]]
    
    if B is None and a is not None and b is not None and A is not None:
        sinB = b * math.sin(A) / a
        return {angles[1]: math.asin(sinB)}, [edges[0], edges[1], angles[0]]
    return {}, []

def fp(U):
    a = U.get("a", None)
    b = U.get("b", None)
    c = U.get("c", None)
    p = U.get("p", None)
    # find p
    if p is None and a is not None and b is not None and c is not None:
        return {"p": (a + b + c) / 2}, ["a", "b", "c"]
    # find a
    if a is None and b is not None and c is not None and p is not None:
        return {"a": (2 * p) / (b + c) / 2}, ["p", "b", "c"]
    # find b
    if b is None and a is not None and c is not None and p is not None:
        return {"b": (2 * p) / (a + c) / 2}, ["p", "a", "c"]
    # find c
    if c is None and a is not None and b is not None and p is not None:
        return {"c": (2 * p) / (a + b) / 2}, ["p", "a", "b"]
    return {}, []

def fsquare_sin(U, edges, angle):
    a = U.get(edges[0], None)
    b = U.get(edges[1], None)
    C = U.get(angle, None)
    S = U.get("S", None)

    # find S
    if a is not None and b is not None and C is not None:
        return {"S": (a * b * math.sin(C)) / 2}, [edges[0], edges[1], angle]
    # find a
    if a is None and b is not None and C is not None and S is not None:
        return {edges[0]: (2 * S) / (b * math.sin(C))}, [edges[1], angle, "S"]
    # find b
    if b is None and a is not None and C is not None and S is not None:
        return {edges[1]: (2 * S) / (a * math.sin(C))}, [edges[0], angle, "S"]
    # find C
    if C is None and a is not None and b is not None and S is not None:
        sinC = (2 * S) / (a * b)
        return {angle: math.asin(sinC)}, [edges[0], edges[1], "S"]
    return {}, []

F = {
    "f1": fangle, 
    "f2": lambda U: fsin(U, ["a", "b"], ["A", "B"]),
    "f3": lambda U: fsin(U, ["b", "c"], ["B", "C"]),
    "f4": lambda U: fsin(U, ["a", "c"], ["A", "C"]),
    "f5": fp,
    "f6": lambda U: fsquare_sin(U, ["a", "b"], "C"),
    "f7": lambda U: fsquare_sin(U, ["a", "c"], "B"),
    "f8": lambda U: fsquare_sin(U, ["b", "c"], "A"),
}
M = ["A", "B", "C", "a", "b", "c", "p", "S"]

### Define hypothesis and goal

In [61]:
H = {
    "A": 0.4,
    "B": 0.3,
    "c": 2.1
}
G = ["S", "p"]

### Algorithm

In [69]:
def find_solution(H, G):
    H = H.copy()
    solution = []
    found = False

    if set(G) <= set(H.keys()):
        return solution, {x: H[x] for x in G}

    while not found:
        Hold = H.copy()
        for f, r in F.items():
            v, u = r(H)
            if len(v) == 0:
                continue

            new_fact = list(v.keys())[0]
            if new_fact not in H.keys():
                H = H | v
                solution.append(f"{f}({','.join(u)}) -> {new_fact}")

        if set(G) <= set(H.keys()):
            found = True
        
        if set(H.keys()) == set(Hold.keys()):
            break
        
    return solution, {x: H[x] for x in G}
print(f"Hypothesis: {H}")
print(f"Goal: {G}")
solution, result = find_solution(H, G)
print(f"Steps: {solution}")
print(f"Result: {result}")

Hypothesis: {'A': 0.4, 'B': 0.3, 'c': 2.1}
Goal: ['S', 'p']
Steps: ['f1(A,B) -> C', 'f3(c,B,C) -> b', 'f4(c,A,C) -> a', 'f5(a,b,c) -> p', 'f6(a,b,C) -> S']
Result: {'S': 0.3938941537385168, 'p': 2.1663702746229285}
